# Criando dataframes a partir de imagens dos dados do Recenseamento de 1920

Para essa apresentação selecionei apenas uma tabela do censo. A tabela foi tirada da página 27 da segunda parte do Volume V que trata dos salários.

O arquivo está salvo no diretório com o nome 'p27.pdf'. 

Primeiro vamos importar os pacotes necessários:

In [1]:
#Ative essa célula caso não tenha o Wand e o pytesseract instalado

# !pip install Wand
# !pip install pytesseract

In [2]:
from wand.image import Image as Img #esse pacote transforma o pdf em um arquivo de imagem
from PIL import Image               #usamos esse pacote para abrir o arquivo de imagem
import pytesseract                  #a função desse pacote é transformar o arquivo de imagem em string
import pandas as pd                 #biblioteca do Python para gerenciar DataFrames
import re                           #será necessário para limpar a string gerada
import csv                          #útil para a leitura do arquivo em csv

O primeiro passo consiste em transformar o arquivo 'p27.pdf' em imagem. Note que o nome do arquivo de imagem gerado é 'image27.jpg'.

In [3]:
with Img(filename='p27.pdf', resolution=400) as img:
    img.compression_quality = 99
    img.save(filename='image27.jpg')

Agora transformamos a imagem em uma string e printamos o resultado. Veja que conseguimos abrir o arquivo, mas ele não está formatado.

In [4]:
im = Image.open('image27.jpg')
text = pytesseract.image_to_string(im, lang = 'eng')
print(text)


XX DIRECTORIA GERAL DE ES'I‘ATISTICA
Comparaqiio do salario médin dos operatios jomaleims, segundo
NUBEERG DE CASGS CONSEDERADGS
()PERARIOS JORNALEIROS
PROFISSEwEs ABUL’HIS ' “5393135
TOTAL M— _
“omens l Mulberes ' Homens Mulheres
1920 1928 1920 l 1928 1920 102-8 ‘ 1920 i 1928 19:20 19:8
Fiagia e Tecidas I I i
Acabador ...................... 613 364 385 303 208 44 J 10 '3 10 1‘1:
Batedor ........................ 171 145 171 14a: 1 ~— __ _ 1 _. __
Branqueador ................... 50 5’9 45 7G *- -— 5 3 -—- } ~—
Cardador ...................... 285 173 241 155: 39 14 6 6 —— ~—
Carreteleiro .................... E 855 S39 78 74. 730 437 37 9 18 19
EngommadOr. . ................ 132 118 126 112? _ -— é — —- -—
Fiandeiro ........ ‘ .............. 2.187 1.872 257 220 924 604 476 585 l 530 463 ‘
Massaroqueim ................. 435 490 279 291 148 199 6 — 2 -—
Meadeim. ..... . ..... , ........ 2m 78 11 1 :2 573? 51 — F - 96 --
Remettedor .................... 331 182 131 119 21 — 166 63 13 m
Tecelﬁ

O resultado acima é a string 'pura'. Vamos agora limpar a string para que ela possa ser transformada em um dataframe.

Começamos retirando o cabeçalho:

In [5]:
text = text.split("\n",9)[-1]
print(text)

Acabador ...................... 613 364 385 303 208 44 J 10 '3 10 1‘1:
Batedor ........................ 171 145 171 14a: 1 ~— __ _ 1 _. __
Branqueador ................... 50 5’9 45 7G *- -— 5 3 -—- } ~—
Cardador ...................... 285 173 241 155: 39 14 6 6 —— ~—
Carreteleiro .................... E 855 S39 78 74. 730 437 37 9 18 19
EngommadOr. . ................ 132 118 126 112? _ -— é — —- -—
Fiandeiro ........ ‘ .............. 2.187 1.872 257 220 924 604 476 585 l 530 463 ‘
Massaroqueim ................. 435 490 279 291 148 199 6 — 2 -—
Meadeim. ..... . ..... , ........ 2m 78 11 1 :2 573? 51 — F - 96 --
Remettedor .................... 331 182 131 119 21 — 166 63 13 m
Tecelﬁo ........................ 64 632 2 .225 3 . 748 . 18% 2 . 60?. 989 90 7.4 192 28
Tintureiro ..................... 553 21? * 518 31?. 1 1 <1 — z —- m
Urdidor ........................ 274 152 143 62 105 73 26 12 l —, .—
Madeiras s
Serrador ....................... 133 i 85 152 85 — — —— — -— -
Torneim em madeira 

Agora utilizamos *regular expressions* para selecionar as linhas que tem o 'perfil' do arquivo

In [6]:
pattern = re.compile(r'[A-z]+\s\.+\s[\w\s]*')
one = pattern.search(text)
one.group()

'Acabador ...................... 613 364 385 303 208 44 J 10 '

Generalizando a expressão para todas as linhas:

In [7]:
f = pattern.findall(text)
f

['Acabador ...................... 613 364 385 303 208 44 J 10 ',
 'Batedor ........................ 171 145 171 14a',
 'Branqueador ................... 50 5',
 'Cardador ...................... 285 173 241 155',
 'Carreteleiro .................... E 855 S39 78 74',
 'Fiandeiro ........ ',
 'Massaroqueim ................. 435 490 279 291 148 199 6 ',
 'Remettedor .................... 331 182 131 119 21 ',
 'o ........................ 64 632 2 ',
 'Tintureiro ..................... 553 21',
 'Urdidor ........................ 274 152 143 62 105 73 26 12 l ',
 'Serrador ....................... 133 i 85 152 85 ',
 'madeira ......... 39 i 24 39 24 ',
 'Ajustador ................. ',
 'Caldeireiro ...... ',
 'Esmerilhador ................. 9 40 7 4O ',
 'er .............. 55 25 55 as ',
 'Ferreiro ....................... 120 15 118 M ',
 'Fundidor ...................... 537 347 435 287 ',
 'Funileiro ..................... 223 f 25 195 1',
 'xLimador ....................... 334 20 279 20 ',
 'Mo

O output do método findall é uma lista. A seguir transformamos essa lista em uma string para dar continuidade a impeza

In [8]:
g =str(f)
g

"['Acabador ...................... 613 364 385 303 208 44 J 10 ', 'Batedor ........................ 171 145 171 14a', 'Branqueador ................... 50 5', 'Cardador ...................... 285 173 241 155', 'Carreteleiro .................... E 855 S39 78 74', 'Fiandeiro ........ ', 'Massaroqueim ................. 435 490 279 291 148 199 6 ', 'Remettedor .................... 331 182 131 119 21 ', 'o ........................ 64 632 2 ', 'Tintureiro ..................... 553 21', 'Urdidor ........................ 274 152 143 62 105 73 26 12 l ', 'Serrador ....................... 133 i 85 152 85 ', 'madeira ......... 39 i 24 39 24 ', 'Ajustador ................. ', 'Caldeireiro ...... ', 'Esmerilhador ................. 9 40 7 4O ', 'er .............. 55 25 55 as ', 'Ferreiro ....................... 120 15 118 M ', 'Fundidor ...................... 537 347 435 287 ', 'Funileiro ..................... 223 f 25 195 1', 'xLimador ....................... 334 20 279 20 ', 'Modelador ............

Substituindo caracteres:

In [9]:
g =g.replace('.','')
g =g.replace(' ',';')
g =g.replace(',','\n')
g =g.replace(';;', ';')
g =g.replace('\n;', '\n')

Podemos ver agora que a string já está em um formato passível de ser lido como dataframe:

In [10]:
g

"['Acabador;613;364;385;303;208;44;J;10;'\n'Batedor;171;145;171;14a'\n'Branqueador;50;5'\n'Cardador;285;173;241;155'\n'Carreteleiro;E;855;S39;78;74'\n'Fiandeiro;'\n'Massaroqueim;435;490;279;291;148;199;6;'\n'Remettedor;331;182;131;119;21;'\n'o;64;632;2;'\n'Tintureiro;553;21'\n'Urdidor;274;152;143;62;105;73;26;12;l;'\n'Serrador;133;i;85;152;85;'\n'madeira;39;i;24;39;24;'\n'Ajustador;'\n'Caldeireiro;'\n'Esmerilhador;9;40;7;4O;'\n'er;55;25;55;as;'\n'Ferreiro;120;15;118;M;'\n'Fundidor;537;347;435;287;'\n'Funileiro;223;f;25;195;1'\n'xLimador;334;20;279;20;'\n'Modelador;62;32;56;22;'\n'Agermmeim;54'\n'Soldador;9'\n'metal;168;46;149;46;'\n'Lapidador;'\n'Vidreiro;397;384;305;243;'\n'Ladrilheiro;55;52;55;52;'\n'Oleiro;304;79;104;79;'\n'Empacotador;290;40;37;'\n'vidms;58;i;22;50;22;'\n'Manipulador;138;22;75;22;59;'\n'veias;56;37;56;3'\n'_;\\nSnboeim;'\n'Bebidas;\\nEugarrafador;cu;arrolhaéor'\n'garrafas;91;63;as;5']"

Salvando a string em format csv

In [11]:
h = open('csvfile.csv','w')
h.write(g)
h.close()

No comando abaixo abrimos o arquivo de csv que acabamos de salvar.

In [12]:
df = pd.read_csv('csvfile.csv', sep= ';', names = 'abcdefghijk', header = None, index_col = False,
                 quoting=csv.QUOTE_NONE)
print(df)


                a               b    c            d     e    f    g    h    i  \
0      ['Acabador             613  364          385   303  208   44    J   10   
1        'Batedor             171  145          171  14a'  NaN  NaN  NaN  NaN   
2    'Branqueador              50   5'          NaN   NaN  NaN  NaN  NaN  NaN   
3       'Cardador             285  173          241  155'  NaN  NaN  NaN  NaN   
4   'Carreteleiro               E  855          S39    78  74'  NaN  NaN  NaN   
5      'Fiandeiro               '  NaN          NaN   NaN  NaN  NaN  NaN  NaN   
6   'Massaroqueim             435  490          279   291  148  199    6    '   
7     'Remettedor             331  182          131   119   21    '  NaN  NaN   
8              'o              64  632            2     '  NaN  NaN  NaN  NaN   
9     'Tintureiro             553  21'          NaN   NaN  NaN  NaN  NaN  NaN   
10       'Urdidor             274  152          143    62  105   73   26   12   
11      'Serrador           

Criamos agora um dataframe apenas com os valores numéricos das colunas de dados

In [13]:
df1 = df.iloc[:,1:11].apply(pd.to_numeric, errors='coerce')
print(df1.head())

       b      c      d      e      f     g   h     i   j   k
0  613.0  364.0  385.0  303.0  208.0  44.0 NaN  10.0 NaN NaN
1  171.0  145.0  171.0    NaN    NaN   NaN NaN   NaN NaN NaN
2   50.0    NaN    NaN    NaN    NaN   NaN NaN   NaN NaN NaN
3  285.0  173.0  241.0    NaN    NaN   NaN NaN   NaN NaN NaN
4    NaN  855.0    NaN   78.0    NaN   NaN NaN   NaN NaN NaN


Concatenando o nome das linhas com os valores:

In [14]:
df_new = pd.concat([df['a'], df1], axis=1)
print(df_new)

                a      b      c      d      e      f      g     h     i   j  \
0      ['Acabador  613.0  364.0  385.0  303.0  208.0   44.0   NaN  10.0 NaN   
1        'Batedor  171.0  145.0  171.0    NaN    NaN    NaN   NaN   NaN NaN   
2    'Branqueador   50.0    NaN    NaN    NaN    NaN    NaN   NaN   NaN NaN   
3       'Cardador  285.0  173.0  241.0    NaN    NaN    NaN   NaN   NaN NaN   
4   'Carreteleiro    NaN  855.0    NaN   78.0    NaN    NaN   NaN   NaN NaN   
5      'Fiandeiro    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN NaN   
6   'Massaroqueim  435.0  490.0  279.0  291.0  148.0  199.0   6.0   NaN NaN   
7     'Remettedor  331.0  182.0  131.0  119.0   21.0    NaN   NaN   NaN NaN   
8              'o   64.0  632.0    2.0    NaN    NaN    NaN   NaN   NaN NaN   
9     'Tintureiro  553.0    NaN    NaN    NaN    NaN    NaN   NaN   NaN NaN   
10       'Urdidor  274.0  152.0  143.0   62.0  105.0   73.0  26.0  12.0 NaN   
11      'Serrador  133.0    NaN   85.0  152.0   85.0

Por fim, colocamos nomes nas colunas e geramos o dataframe completo.

In [15]:
cols = ['Profissão', '1920', '1928', '20_A_H', '28_A_H','20_A_M', '28_A_M', '20_M_H', '28_M_H','20_M_M', '28_M_M']

In [16]:
df_new.columns = cols
print(df_new)

        Profissão   1920   1928  20_A_H  28_A_H  20_A_M  28_A_M  20_M_H  \
0      ['Acabador  613.0  364.0   385.0   303.0   208.0    44.0     NaN   
1        'Batedor  171.0  145.0   171.0     NaN     NaN     NaN     NaN   
2    'Branqueador   50.0    NaN     NaN     NaN     NaN     NaN     NaN   
3       'Cardador  285.0  173.0   241.0     NaN     NaN     NaN     NaN   
4   'Carreteleiro    NaN  855.0     NaN    78.0     NaN     NaN     NaN   
5      'Fiandeiro    NaN    NaN     NaN     NaN     NaN     NaN     NaN   
6   'Massaroqueim  435.0  490.0   279.0   291.0   148.0   199.0     6.0   
7     'Remettedor  331.0  182.0   131.0   119.0    21.0     NaN     NaN   
8              'o   64.0  632.0     2.0     NaN     NaN     NaN     NaN   
9     'Tintureiro  553.0    NaN     NaN     NaN     NaN     NaN     NaN   
10       'Urdidor  274.0  152.0   143.0    62.0   105.0    73.0    26.0   
11      'Serrador  133.0    NaN    85.0   152.0    85.0     NaN     NaN   
12       'madeira   39.0 

Se abrirmos o arquivo original veremos que alguns dados que aparecem nele não estão disponíveis nesse dataframe final. Isso acontece por que o Python não consegue identificar todos os números. Obviamente, poderíamos melhorar a resolução da captura da imagem, mas é improvável que isso reduza a zero o volume de erros já que alguns números são difíceis de se observar mesmo a olho nu.